# Are All Tipping Points Predictable? A Test of Early Warning Signal Theory

## Objective
To evaluate whether early warning signals (EWS) such as rising autocorrelation (AR1) and variance can reliably predict abrupt shifts (tipping points) in palaeoclimate data.

## Input Data
- Proxy Dataset: NGRIP d18O values (dataset.txt)
- Target Events: Younger Dryas End, 8.2k Event, Holocene Thermal Maximum Onset

## Methodology
1. Time Series Slicing: Windowed subsets extracted around each tipping point (+/- 2000 years).
2. Detrending Methods: Gaussian smoothing, Savitzky-Golay filter, Spline fitting, First differencing.
3. EWS Metrics: Lag-1 autocorrelation (AR1), Variance, Kendall's tau.
4. Significance Testing: Phase-randomized surrogates (300 iterations).

## Results Summary
- Younger Dryas: Strong signals but sensitive to window size.
- 8.2k Event: No consistent signal.
- HTM Onset: Robust autocorrelation signal.

In [ ]:
import os
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.ndimage import gaussian_filter1d
from scipy.signal import savgol_filter
from scipy.interpolate import UnivariateSpline
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from IPython.display import display
import warnings

# CONFIGURATION
DATA_PATH     = "dataset.txt"
OUTPUT_DIR    = "Figures"
WINDOW_AROUND = 2000
EVENT_AGES    = {
    'Younger_Dryas_End':             11700,
    '8.2k_Event':                     8200,
    'Holocene_Thermal_Maximum_Onset': 6500,
}
DETREND_METHODS = ['gaussian','savgol','spline','diff']
WINDOW_PCTS     = [25,35,45,55,65,75]
N_SURR          = 300
DPI             = 300

# PLOTTING & FONT CONFIG
sns.set(style="whitegrid")
TITLE_FONTSIZE = 10
LABEL_FONTSIZE = 8
TICK_FONTSIZE  = 7
LEGEND_FONTSIZE= 7
ANNOT_FONTSIZE = 8

# FIGURE SIZES
FIG1_SIZE      = (8, 4)
FIG2_SIZE      = (8, 6)
FIG3_SIZE      = (5, 4)
FIG4_SIZE      = (7, 4)
FIG5_SIZE      = (8, 4)
HEATMAP_FIGSIZE= (6, 5)

# HELPER FUNCTIONS
def ensure_dir(path): os.makedirs(path, exist_ok=True)
def load_and_sort(path):
    df = pd.read_csv(path, sep='\t', comment='#'); time_full=df['age_calBP'].values; proxy_full=df['d18O_vsmow'].values
    return time_full, proxy_full
def detrend_series(ts, method, window_len=101, polyorder=3):
    L = len(ts)
    if method=='gaussian': return ts - gaussian_filter1d(ts, sigma=window_len/6)
    if method=='savgol': wl = min(window_len, L if L%2 else L-1); wl = max(wl,3); return ts - savgol_filter(ts, window_length=wl, polyorder=min(polyorder, wl-1))
    if method=='spline': x=np.arange(L); return ts - UnivariateSpline(x, ts, s=L)(x)
    if method=='diff': return np.diff(ts, prepend=ts[0])
    raise ValueError
def rolling_stats(ts, win):
    s=pd.Series(ts); return s.rolling(win,center=True).var().to_numpy(), s.rolling(win,center=True).apply(lambda x: x.autocorr(lag=1), raw=False).to_numpy()
def kendall_tau(x,y): return stats.kendalltau(x,y,nan_policy='omit').correlation
def phase_surrogates(ts, n=N_SURR):
    N=len(ts); fft0=np.fft.rfft(ts); A,phi=np.abs(fft0),np.angle(fft0); out=[]
    for _ in range(n): rnd=np.random.uniform(0,2*np.pi,size=phi.shape); rnd[0]=phi[0]; out.append(np.fft.irfft(A*np.exp(1j*rnd),n=N))
    return np.array(out)

# MAIN SCRIPT
if __name__ == '__main__':
    warnings.filterwarnings("ignore", category=FutureWarning)
    ensure_dir(OUTPUT_DIR)
    time_full, proxy_full = load_and_sort(DATA_PATH)

    events = {}
    for name, age0 in EVENT_AGES.items():
        mask = (time_full >= age0-WINDOW_AROUND) & (time_full <= age0+WINDOW_AROUND)
        events[name] = {'time': time_full[mask] - age0, 'series': proxy_full[mask]}

    # Parameter Sweep & Summary Table Generation
    print("Performing parameter sweep and significance testing")
    recs=[]
    for ev,dat in tqdm(events.items(), desc="Events"):
        t,ts=dat['time'],dat['series']
        for method in DETREND_METHODS:
            dts=detrend_series(ts,method)
            surs=phase_surrogates(dts)
            for pct in WINDOW_PCTS:
                w=max(3,int(len(dts)*pct/100))
                var_ts, ac_ts = rolling_stats(dts,w)
                τ_var=kendall_tau(t,var_ts)
                τ_ac =kendall_tau(t,ac_ts)
                sτ_ac = [kendall_tau(t, rolling_stats(s,w)[1]) for s in surs]
                p_emp = (np.sum(np.array(sτ_ac)>=τ_ac)+1)/(len(sτ_ac)+1)
                recs.append({"Event":ev, "Detrend":method, "WindowPct":pct, "TauVar":τ_var, "TauAC":τ_ac, "AC_pValue":p_emp})
    summary_df = pd.DataFrame(recs)
    summary_df.to_csv(f"{OUTPUT_DIR}/ews_summary_table.csv", index=False)
    print("Summary table generated and saved.")
    print("-" * 50)

    # PLOTTING
    print("Generating final, polished figures")
    colors = {'Younger_Dryas_End': 'blue', '8.2k_Event': 'green', 'Holocene_Thermal_Maximum_Onset': 'red'}

    # Proxy comparison
    plt.figure(figsize=FIG1_SIZE)
    for ev,col in zip(events, colors.values()): plt.plot(events[ev]['time'], events[ev]['series'], label=ev, color=col)
    plt.axvline(0, color='k', linestyle='--', label='Tipping Point'); plt.gca().invert_xaxis()
    plt.xlabel("Time to Transition (yr)", fontsize=LABEL_FONTSIZE); plt.ylabel("δ¹⁸O (‰)", fontsize=LABEL_FONTSIZE)
    plt.title("Proxy Comparison", fontsize=TITLE_FONTSIZE) 
    plt.xticks(fontsize=TICK_FONTSIZE); plt.yticks(fontsize=TICK_FONTSIZE)
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), ncol=4, fontsize=LEGEND_FONTSIZE); plt.tight_layout(rect=[0, 0.1, 1, 1])
    plt.savefig(f"{OUTPUT_DIR}/Proxy_Comparison.png", dpi=DPI); plt.close()

    # Rolling variance and AC
    fig, axs = plt.subplots(2, 1, figsize=FIG2_SIZE, sharex=True)
    for ev, col in zip(events, colors.values()):
        t, ts = events[ev]['time'], events[ev]['series']
        dts = detrend_series(ts, 'gaussian')
        var, ac = rolling_stats(dts, win=int(0.5 * len(dts)))
        axs[0].plot(t, var, color=col, label=ev)
        axs[1].plot(t, ac, color=col, label=ev)
    
    for ax, lab in zip(axs, ["Rolling Variance", "Rolling Autocorrelation (lag-1)"]):
        ax.axvline(0, color='k', linestyle='--')
        ax.set_ylabel(lab, fontsize=LABEL_FONTSIZE)
        ax.tick_params(axis='both', which='major', labelsize=TICK_FONTSIZE)
    
    axs[1].set_xlabel("Time to Transition (yr)", fontsize=LABEL_FONTSIZE)
    fig.suptitle("Rolling EWS Analysis", fontsize=TITLE_FONTSIZE)
    handles, labels = axs[1].get_legend_handles_labels()
    fig.legend(handles, labels, loc='lower center', ncol=3, fontsize=LEGEND_FONTSIZE, bbox_to_anchor=(0.5, 0.01))
    plt.subplots_adjust(bottom=0.15)
    plt.savefig(f"{OUTPUT_DIR}/Rolling_EWS.png", dpi=DPI)
    plt.close()
    

    # Distribution shift
    for ev in events:
        t, ts = events[ev]['time'], events[ev]['series']; dts = detrend_series(ts, 'gaussian'); before = dts[t<0]; during = dts[(t>=0)&(t<=500)]
        lev_p = stats.levene(before,during).pvalue; ks_p  = stats.ks_2samp(before,during).pvalue
        plt.figure(figsize=FIG3_SIZE); sns.kdeplot(before, fill=True, label='Before'); sns.kdeplot(during, fill=True, label='During')
        plt.title(f"Distribution Shift for {ev}\nLevene p={lev_p:.3f}, KS p={ks_p:.3f}", fontsize=TITLE_FONTSIZE) 
        plt.xlabel("Detrended δ¹⁸O", fontsize=LABEL_FONTSIZE); plt.ylabel("Density", fontsize=LABEL_FONTSIZE)
        plt.xticks(fontsize=TICK_FONTSIZE); plt.yticks(fontsize=TICK_FONTSIZE)
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.18), ncol=2, fontsize=LEGEND_FONTSIZE); plt.tight_layout(rect=[0, 0.1, 1, 1])
        plt.savefig(f"{OUTPUT_DIR}/DistShift_{ev}.png", dpi=DPI); plt.close()

    # Sensitivity to window size
    plt.figure(figsize=FIG4_SIZE)
    for ev,col in zip(events, colors.values()):
        taus=[]; t,ts=events[ev]['time'],events[ev]['series']; dts=detrend_series(ts,'gaussian')
        for pct in WINDOW_PCTS:
            ac=rolling_stats(dts,int(len(dts)*pct/100))[1]; taus.append(kendall_tau(t,ac))
        plt.plot(WINDOW_PCTS,taus,'-o',label=ev,color=col)
    plt.axhline(0,color='k',linestyle='--'); plt.xlabel("Window Size (%)", fontsize=LABEL_FONTSIZE); plt.ylabel("Kendall’s τ", fontsize=LABEL_FONTSIZE)
    plt.title("Sensitivity to Window Size", fontsize=TITLE_FONTSIZE); plt.xticks(fontsize=TICK_FONTSIZE); plt.yticks(fontsize=TICK_FONTSIZE) 
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=3, fontsize=LEGEND_FONTSIZE); plt.tight_layout(rect=[0, 0.1, 1, 1])
    plt.savefig(f"{OUTPUT_DIR}/Sensitivity_WindowSize.png", dpi=DPI); plt.close()

    # Detrending robustness
    plt.figure(figsize=FIG5_SIZE)
    for method,color in zip(['gaussian','diff'], ['blue','orange']):
        taus=[]; t = events['Holocene_Thermal_Maximum_Onset']['time']; ts= events['Holocene_Thermal_Maximum_Onset']['series']
        dts = detrend_series(ts,method)
        for pct in WINDOW_PCTS:
            ac  = rolling_stats(dts,int(len(dts)*pct/100))[1]; taus.append(kendall_tau(t,ac))
        plt.plot(WINDOW_PCTS, taus,'-o',label=method,color=color)
    plt.axhline(0,color='k',linestyle='--'); plt.xlabel("Window Size (%)", fontsize=LABEL_FONTSIZE); plt.ylabel("Kendall’s τ", fontsize=LABEL_FONTSIZE)
    plt.title("Detrending Robustness", fontsize=TITLE_FONTSIZE); plt.xticks(fontsize=TICK_FONTSIZE); plt.yticks(fontsize=TICK_FONTSIZE) 
    plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.2), ncol=2, fontsize=LEGEND_FONTSIZE); plt.tight_layout(rect=[0, 0.1, 1, 1])
    plt.savefig(f"{OUTPUT_DIR}/Detrending_Robustness.png", dpi=DPI); plt.close()

    # Heatmaps
    for ev in EVENT_AGES:
        df_ev = summary_df[summary_df.Event == ev]
        hm_data = df_ev.pivot(index='Detrend', columns='WindowPct', values='TauAC')
        
        plt.figure(figsize=HEATMAP_FIGSIZE)
        ax = sns.heatmap(hm_data, annot=False, fmt=".2f", center=0, cmap="vlag", vmin=-1.0, vmax=1.0) 
        for i in range(hm_data.shape[0]):
            for j in range(hm_data.shape[1]):
                value = hm_data.iloc[i, j]
                if not pd.isna(value):
                    ax.text(j + 0.5, i + 0.5, f'{value:.2f}',
                            ha='center', va='center', color='white' if abs(value) > 0.6 else 'black',
                            fontsize=ANNOT_FONTSIZE)
    
        plt.title(f"Heatmap of Tau(AC) for {ev}", fontsize=TITLE_FONTSIZE)
        plt.ylabel("Detrend Method", fontsize=LABEL_FONTSIZE)
        plt.xlabel("Window Size (%)", fontsize=LABEL_FONTSIZE)
        plt.xticks(fontsize=TICK_FONTSIZE, rotation=0)
        plt.yticks(fontsize=TICK_FONTSIZE, rotation=0)
        plt.tight_layout()
        plt.savefig(f"{OUTPUT_DIR}/Heatmap_TauAC_{ev}.png", dpi=DPI)
        plt.close()

    print("All figures generated with final layout and font adjustments")
    print("-" * 50)
    print(f"\nAll figures and summary table saved to folder: '{os.path.abspath(OUTPUT_DIR)}'")

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ks_2samp, levene
from tqdm import tqdm

# Loading the data
file_path = 'dataset.txt'
df = pd.read_csv(file_path, sep='\t')
df = df.rename(columns={'age_calBP': 'Age_BP', 'd18O_vsmow': 'D18O'})

# Extracting "Before" and "During" windows for each event
events_data = {
    'Younger Dryas': {
        'Before': df[(df['Age_BP'] >= 12200) & (df['Age_BP'] < 13200)]['D18O'].values,
        'During': df[(df['Age_BP'] >= 11700) & (df['Age_BP'] < 12200)]['D18O'].values
    },
    '8.2k Event': {
        'Before': df[(df['Age_BP'] >= 8200) & (df['Age_BP'] < 9200)]['D18O'].values,
        'During': df[(df['Age_BP'] >= 7700) & (df['Age_BP'] < 8200)]['D18O'].values
    },
    'Holocene Thermal Maximum': {
        'Before': df[(df['Age_BP'] >= 9200) & (df['Age_BP'] < 10200)]['D18O'].values,
        'During': df[(df['Age_BP'] >= 8700) & (df['Age_BP'] < 9200)]['D18O'].values
    }
}

# Bootstrap Analysis
n_boot = 1000
rng = np.random.default_rng(42)

results = []

for event_name, data in events_data.items():
    before = data['Before']
    during = data['During']
    
    ks_stat_real, ks_p_real = ks_2samp(before, during)
    lev_stat_real, lev_p_real = levene(before, during)
    
    ks_stats = []
    lev_stats = []

    for _ in tqdm(range(n_boot), desc=f"Bootstrapping {event_name}"):
        b_boot = rng.choice(before, size=len(before), replace=True)
        d_boot = rng.choice(during, size=len(during), replace=True)

        ks_stat, _ = ks_2samp(b_boot, d_boot)
        lev_stat, _ = levene(b_boot, d_boot)
        
        ks_stats.append(ks_stat)
        lev_stats.append(lev_stat)
    
  
    ks_ci = np.percentile(ks_stats, [2.5, 97.5])
    lev_ci = np.percentile(lev_stats, [2.5, 97.5])
    ks_p_boot = np.mean(np.array(ks_stats) >= ks_stat_real)
    lev_p_boot = np.mean(np.array(lev_stats) >= lev_stat_real)

    results.append({
        'Event': event_name,
        'K-S Stat (Real)': round(ks_stat_real, 3),
        'K-S 95% CI': f"{ks_ci[0]:.3f} – {ks_ci[1]:.3f}",
        'K-S p (Boot)': round(ks_p_boot, 3),
        'Levene Stat (Real)': round(lev_stat_real, 3),
        'Levene 95% CI': f"{lev_ci[0]:.3f} – {lev_ci[1]:.3f}",
        'Levene p (Boot)': round(lev_p_boot, 3)
    })

# Printing results as a table
bootstrap_df = pd.DataFrame(results)

In [ ]:
bootstrap_df